In [ ]:
import os
data_paths['test'] = '/kaggle/input/intel-image-classification/seg_test/seg_test'
class_names = sorted(os.listdir(data_paths['test']))
print(" class_names:", class_names)


 class_names: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


Bu kontrolü yapmamızın amacı, **test veri setinde hangi sınıfların bulunduğunu ve sıralarını doğrulamaktır.**

`os.listdir(data_paths['test'])` ile test klasöründeki tüm alt klasörleri listeledik. Her alt klasör bir sınıfı temsil ediyor (buildings, forest, vb.).

`sorted()` ile bu sınıf isimlerini alfabetik olarak sıraladık, böylece modelin tahminleri ve confusion matrix analizleri sırasında sınıf sıralamasıyla uyuşmasını sağladık.

Bu sayede, test setiyle çalışırken sınıf isimlerini yanlış eşleştirme riskini ortadan kaldırmış olduk ve analizlerimizin doğru sınıf bazlı olmasını garantiledik.

In [ ]:

# Son Conv2D katman ismi (modelinize göre değiştirin)
LAST_CONV_LAYER_NAME = "conv2d_320"  # Modelinizdeki son conv layer adı bu örnekte

def make_gradcam_heatmap(img_array, model, last_conv_layer_name=LAST_CONV_LAYER_NAME, pred_index=None):
    try:
        # Modelin son conv katmanı ve çıktılarını almak için alt model oluştur
        grad_model = tf.keras.models.Model(
            [model.inputs],
            [model.get_layer(last_conv_layer_name).output, model.output]
        )

        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(predictions[0])
            class_channel = predictions[:, pred_index]

        grads = tape.gradient(class_channel, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        conv_outputs = conv_outputs[0]
        heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)

        # Heatmap negatif değerleri 0 yap ve normalize et
        heatmap = tf.maximum(heatmap, 0)
        max_val = tf.math.reduce_max(heatmap)
        if max_val == 0:
            return np.zeros(heatmap.shape)
        heatmap /= max_val

        return heatmap.numpy()

    except Exception as e:
        print(f"Grad-CAM hesaplama hatası: {e}")
        return np.zeros((224, 224))


def display_gradcam_with_prediction(model, img_path, class_names, img_size=(224, 224), last_conv_layer_name=LAST_CONV_LAYER_NAME):
    # Görüntüyü yükle ve modelin beklediği boyuta getir
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize

    preds = model.predict(img_array)
    pred_class = np.argmax(preds[0])
    confidence = preds[0][pred_class]

    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name=last_conv_layer_name, pred_index=pred_class)

    # Heatmap kontrolü ve düzeltme
    if heatmap.max() == 0:
        print(f" Heatmap boş, rastgele heatmap oluşturuluyor...")
        heatmap = np.random.rand(*heatmap.shape) * 0.5
    
    # Heatmap'i normalize et (0-1 arası)
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
    
    # Gaussian blur uygula (daha yumuşak görünüm için)
    from scipy.ndimage import gaussian_filter
    heatmap_smooth = gaussian_filter(heatmap, sigma=0.5)
    
    # Heatmap'i orijinal görüntü boyutuna büyüt (smooth interpolation)
    from scipy.ndimage import zoom
    scale_factor = img_size[0] / heatmap_smooth.shape[0], img_size[1] / heatmap_smooth.shape[1]
    heatmap_resized = zoom(heatmap_smooth, scale_factor, order=1)  # Bilinear interpolation
    
    # Renk haritası için uint8'e çevir
    heatmap_uint8 = np.uint8(255 * heatmap_resized)
    
    # Renk haritası uygula
    heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)

    # Orijinal görüntü
    original_img = np.array(img).astype(np.uint8)
    
    # Superimpose - alpha blending
    alpha = 0.4  # Heatmap şeffaflığı
    superimposed_img = np.uint8((1-alpha) * original_img + alpha * heatmap_colored)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.title("Orijinal Görüntü", fontweight='bold')
    plt.imshow(original_img)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title("Grad-CAM Heatmap", fontweight='bold')
    plt.imshow(heatmap_resized, cmap='jet', interpolation='bicubic', alpha=0.8)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title(f"Dikkat Haritası\n{class_names[pred_class]} ({confidence:.3f})", fontweight='bold')
    plt.imshow(superimposed_img)
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    print(f"Görüntü: {os.path.basename(img_path)}")
    print(f"Tahmin: {class_names[pred_class]} - Güven: {confidence:.4f}")
    print(f"Heatmap orijinal boyut: {heatmap.shape}, yeniden boyutlandırıldı: {heatmap_resized.shape}")
    print("-" * 60)


def run_gradcam_on_all_images(model, data_dir, class_names, img_size=(224, 224), max_images_per_class=5, last_conv_layer_name=LAST_CONV_LAYER_NAME):
    for class_name in class_names:
        class_path = os.path.join(data_dir, class_name)
        if not os.path.exists(class_path):
            print(f" Klasör bulunamadı: {class_path}")
            continue

        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not images:
            print(f" {class_name} sınıfında görüntü yok.")
            continue

        print(f" {class_name} sınıfından örnekler:")
        count = 0
        for img_file in images:
            img_path = os.path.join(class_path, img_file)
            display_gradcam_with_prediction(model, img_path, class_names, img_size=img_size, last_conv_layer_name=last_conv_layer_name)

            count += 1
            if count >= max_images_per_class:
                break


# ----------- KULLANIM ------------

# Modelinizi oluşturun veya yükleyin
# Örnek:
model = create_advanced_cnn_model()
model.load_weights('/kaggle/working/best_intel_model.h5')

# Sınıf isimleri
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

# Test veri klasör yolu
test_data_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test'

# Grad-CAM'ı çalıştırın
run_gradcam_on_all_images(
    model, 
    test_data_dir, 
    class_names, 
    img_size=(224, 224), 
    max_images_per_class=3, 
    last_conv_layer_name=LAST_CONV_LAYER_NAME
)

Bu kod parçası, eğitimli bir CNN modelinin sınıflandırma kararlarını hangi bölgelere dikkat ederek verdiğini görselleştirmek için **Grad-CAM yöntemi**ni kullanır. Amaç, modelin tahmin ettiği sınıf için dikkat haritası oluşturarak bunu orijinal görüntüyle birleştirmektir.

Kodun işleyişi şu şekildedir: Öncelikle Grad-CAM için modelin son Conv2D katmanı belirlenir. Bu katmanın çıktısı ve modelin tahminleri kullanılarak bir alt model oluşturulur. Daha sonra, seçilen sınıfa ait tahminin gradyanları hesaplanır ve bu gradyanlarla katman çıktısı ağırlıklandırılarak bir heatmap elde edilir. Heatmap negatif değerler için sıfırlanır ve normalize edilir. Görselleştirme aşamasında heatmap Gaussian blur ile yumuşatılır, orijinal görüntüyle birleştirilir ve üçlü bir görsel olarak gösterilir: orijinal görüntü, heatmap ve dikkat haritası (orijinal + heatmap).

Tüm sınıflardan belirli sayıda örnek için Grad-CAM çalıştırılır ve her görüntü için tahmin, güven skoru ve heatmap görselleştirilir.

Çıktıya bakıldığında, kod hata vermektedir çünkü Grad-CAM için belirtilen son Conv2D katmanı (conv2d_320) modelde bulunmamaktadır. Mevcut katmanlar conv2d_440, conv2d_441 vb. olduğundan, gerçek heatmap yerine rastgele heatmap üretilmektedir. Tahminler doğru sınıfları göstermekte ve güven skorları da hesaplanmaktadır. Heatmap boyutları orijinal görüntüyle aynı olacak şekilde ayarlanmıştır.

Özetle, kod doğru çalışmakta ancak son Conv2D katman adı yanlış olduğu için Grad-CAM gerçek heatmap’i üretememektedir. Katman adı düzeltilirse, modelin görüntü üzerindeki dikkat bölgeleri net bir şekilde görülebilir. Bu sorun üzerinde çok dursam da çözebilmiş değilim. Elimden bu kadar geldi.